In [ ]:
# For files e data manipulations
# from neuralNetwork import *
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
          
import zipfile                
import shutil            
import glob         

#For math operations           
import time               
import random               

#For images operations
import cv2             
import pydicom          
import scipy
from PIL import Image     
from numpy import expand_dims  


import tensorflow as tf
import keras     
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout,BatchNormalization,Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.optimizers import Adam


pip install -r requirements.txt

In [ ]:
# Version	        Python version	Compiler	Build tools	     cuDNN	CUDA

# tensorflow-2.3.0	3.5-3.8	         GCC 7.3.1	 Bazel 3.1.0	 7.6	 10.1

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:12:52_Pacific_Daylight_Time_2019
Cuda compilation tools, release 10.1, V10.1.243


In [8]:
import tensorflow as tf

# Check if there are any GPUs available
physical_devices = tf.config.list_physical_devices('GPU')

if len(physical_devices) == 0:
    print("No GPU devices found.")
else:
    print("Available GPU devices:")
    for device in physical_devices:
        print(device)


No GPU devices found.


In [ ]:
# Check if GPU is available and visible to TensorFlow
physical_devices = tf.config.list_physical_devices('GPU')

if len(physical_devices) == 0:
    raise RuntimeError("No GPU devices found.")

# Print information about the GPU(s)
for device in physical_devices:
    print("Device name:", device.name)

# Optional: Limit GPU memory growth
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

# Your TensorFlow code here...


## Neural Network

In [ ]:
pnn = PreNeuralNetwork()

In [ ]:
dir_train = 'C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/train'
dir_test  = 'C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/test'
dir_valid = 'C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/validation'


batch_size = 64
width = 512
length = 512
channel = 3
image_size = (width, length)
random.seed(123) 

Como está dando o CLOCK_WATCHDOG_TIMEOUT reduzirei a quantidade de imagens

In [ ]:
print(f'Total training  {len(os.listdir(dir_train))}')
print(f'Total validation {len(os.listdir(dir_valid))}')
print(f'Total test {len(os.listdir(dir_test))}')

In [ ]:
train_datagen= ImageDataGenerator(rescale=1.0/255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,)
val_datagen  = ImageDataGenerator(rescale=1.0/255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,)
test_datagen = ImageDataGenerator(rescale=1.0/255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,)

train_generator=train_datagen.flow_from_directory(dir_train,
                                        target_size=image_size,
                                        color_mode='grayscale',  # Ensure RGB color mode
                                        class_mode='categorical',  # Adjust class_mode as needed
                                        batch_size=batch_size,
                                        subset='training',  # Specify 'training' for the training set
                                        
                                    )

val_generator=val_datagen.flow_from_directory(dir_valid,
                                        target_size=image_size,
                                        color_mode='grayscale',
                                        class_mode='categorical',
                                        batch_size=batch_size
                                    )

test_generator=test_datagen.flow_from_directory(dir_test,
                                        target_size=image_size,
                                        color_mode='grayscale',
                                        class_mode='categorical',
                                        batch_size=batch_size
                                    )

In [ ]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Specify GPU device
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


if tf.test.is_gpu_available():
    print("GPU is available and accessible.")
else:
    print("GPU is not available.")

In [ ]:
n_classes = train_generator.num_classes
print("Number of classes:", n_classes)

In [ ]:
# For files e data manipulations
from neuralNetwork import *

In [ ]:

# image_shape = (width, length, channel)  # Adjust image dimensions as needed

# B7 = EfficientNetModelB7(n_classes, image_shape)
# # Train the model
# modelB7 = B7.build_model()

In [ ]:
# with tf.device('/device:GPU:0'):
#     historyB7 = B7.train(train_generator, val_generator, modelB7, epochs=20, batch_size=batch_size, early_stop_patience=30)

In [ ]:
65165svsvdsfzv

In [ ]:
# image_shape = (width, length, channel) 
image_shape = (128, 128, channel) 

base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=image_shape)

output = base_model.output
output = GlobalAveragePooling2D()(base_model.output)
output = Dense(1024, activation='relu')(output)
output = BatchNormalization()(output)

    
predictions = Dense(n_classes, activation='softmax')(output)
modelB7 = Model(inputs=base_model.input, outputs=predictions)
modelB7.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stop_callback = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

with tf.device('/device:GPU:0'):
    # historyB7 = B7.train(train_generator, val_generator, modelB7, epochs=20, batch_size=batch_size, early_stop_patience=30)
    history = modelB7.fit(
        train_generator,
        validation_data=val_generator,
        epochs=20,
        batch_size=2#,
        #callbacks=[early_stop_callback]
    )

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_gen)

In [ ]:
df_results = model.create_metrics_dataframe(history_b7)
df_results

In [ ]:
model.plot_history(history_b7)

In [ ]:
model = ResNet101Model(num_classes, image_shape)

# Train the model
history_101 = model.train(train_generator, val_generator, epochs=20, batch_size=batch_size, early_stop_patience=5)

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
df_results = model.create_metrics_dataframe(history_101)
model.plot_history(history_101)

In [ ]:
df_results